# Governance Voting Data into Pandas Dataframes Parser

**[Johnnatan Messias](https://johnnatan-messias.github.io/), March 2025**

This notebook file parses the governance voting data into Pandas dataframes. The data is available is then stored in the `./data/` folder.


In [1]:
import os
import gzip
import pickle
import json
import pandas as pd

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
from utils import (proposal_created_to_dataframe,
                   proposal_queued_to_dataframe, proposal_executed_to_dataframe,
                   proposal_cancelled_to_dataframe, vote_cast_to_dataframe,
                   proposal_voting_delay_to_dataframe, voting_delay_set_to_dataframe,
                   approval_to_dataframe, delegate_changed_to_dataframe,
                   proposal_threshold_set_to_dataframe, voting_period_set_to_dataframe,
                   delegate_votes_changed_to_dataframe, transfer_to_dataframe,
                   minter_changed_to_dataframe, new_implementation_to_dataframe)

In [4]:
def get_func(event_name):
    if event_name == 'Approval':
        return approval_to_dataframe
    elif event_name == 'DelegateChanged':
        return delegate_changed_to_dataframe
    elif event_name == 'DelegateVotesChanged':
        return delegate_votes_changed_to_dataframe
    elif event_name == 'Transfer':
        return transfer_to_dataframe
    elif event_name == 'MinterChanged':
        return minter_changed_to_dataframe
    elif event_name == 'ProposalExecuted':
        return proposal_executed_to_dataframe
    elif event_name == 'ProposalQueued':
        return proposal_queued_to_dataframe
    elif event_name == 'ProposalCanceled':
        return proposal_cancelled_to_dataframe
    elif event_name == 'ProposalVotingDelay':
        return proposal_voting_delay_to_dataframe
    elif event_name == 'VotingPeriodSet':
        return voting_period_set_to_dataframe
    elif event_name == 'ProposalCreated':
        return proposal_created_to_dataframe
    elif event_name == 'VoteCast':
        return vote_cast_to_dataframe
    elif event_name == 'NewImplementation':
        return new_implementation_to_dataframe
    elif event_name == 'ProposalThresholdSet':
        return proposal_threshold_set_to_dataframe
    elif event_name == 'VotingDelaySet':
        return voting_delay_set_to_dataframe
    else:
        raise ValueError(f"Event {event_name} not recognized")

In [5]:
# Set directory paths
data_dir = os.path.abspath(
    os.path.join(os.getcwd(), "..", "data")) + os.sep

# Create directories if they don't exist
os.makedirs(data_dir, exist_ok=True)

In [6]:
contract_names = ['compound-governor-alpha',
                  'compound-governor-bravo', 'uniswap-governor-bravo']

In [7]:
def load_dataset(contract_name):
    file_dir = os.path.join(data_dir, 'events_' + contract_name + '.pkl.gz')
    with gzip.open(file_dir, 'rb') as f:
        events = pickle.load(f)
    return events

In [8]:
# Load all contract events from contract
events = dict()
for contract_name in contract_names:
    print(f"Loading events for {contract_name}")
    events[contract_name] = load_dataset(contract_name)

Loading events for compound-governor-alpha
Loading events for compound-governor-bravo
Loading events for uniswap-governor-bravo


In [9]:
for contract_name in events:
    print(f"{contract_name} has {len(events[contract_name])} distinct events")
    for event_name, event in events[contract_name].items():
        print("\t{} has {} registers".format(event_name, len(event)))

compound-governor-alpha has 5 distinct events
	ProposalCanceled has 2 registers
	ProposalCreated has 42 registers
	ProposalExecuted has 36 registers
	ProposalQueued has 36 registers
	VoteCast has 2416 registers
compound-governor-bravo has 11 distinct events
	NewAdmin has 0 registers
	NewImplementation has 5 registers
	NewPendingAdmin has 0 registers
	ProposalCanceled has 29 registers
	ProposalCreated has 265 registers
	ProposalExecuted has 202 registers
	ProposalQueued has 210 registers
	ProposalThresholdSet has 2 registers
	VoteCast has 12425 registers
	VotingDelaySet has 1 registers
	VotingPeriodSet has 1 registers
uniswap-governor-bravo has 11 distinct events
	NewAdmin has 0 registers
	NewImplementation has 1 registers
	NewPendingAdmin has 0 registers
	ProposalCanceled has 22 registers
	ProposalCreated has 67 registers
	ProposalExecuted has 43 registers
	ProposalQueued has 43 registers
	ProposalThresholdSet has 1 registers
	VoteCast has 51580 registers
	VotingDelaySet has 0 register

## Loading Block Timestampps


In [10]:
filename = "block_timestamp__9600000_20563001.csv.gz"
file_dir = os.path.join(data_dir, "blocks", filename)
blocks_df = pd.read_csv(file_dir, compression='gzip')
print("There are {} blocks in the blocks dataset".format(len(blocks_df)))
blocks_df['timestamp'] = pd.to_datetime(
    blocks_df['timestamp']).dt.tz_localize(None)
blocks_df.head()

There are 10963002 blocks in the blocks dataset


,number,timestamp
0,20563001,2024-08-19 13:34:11
1,20563000,2024-08-19 13:33:59
2,20562999,2024-08-19 13:33:47
3,20562998,2024-08-19 13:33:35
4,20562997,2024-08-19 13:33:23


# Governance Dataset Parser


## Compound Governance


### Compound Vote Cast


In [11]:
# Convert the VoteCast event data to dataframe
alpha_vote_df = vote_cast_to_dataframe(
    events['compound-governor-alpha']['VoteCast'])
bravo_vote_df = vote_cast_to_dataframe(
    events['compound-governor-bravo']['VoteCast'])
alpha_vote_df['governor'] = 'alpha'
bravo_vote_df['governor'] = 'bravo'
print("Vote cast")
print(
    f"\tAlpha: Min blockNumber: {alpha_vote_df['blockNumber'].min()} and Max blockNumber: {alpha_vote_df['blockNumber'].max()}")
print(
    f"\tBravo: Min blockNumber: {bravo_vote_df['blockNumber'].min()} and Max blockNumber: {bravo_vote_df['blockNumber'].max()}")
votes_df = pd.concat([alpha_vote_df, bravo_vote_df])
votes_df.sort_values(
    by=['blockNumber', 'transactionIndex', 'proposalId'], inplace=True)
del (alpha_vote_df)
del (bravo_vote_df)
votes_df = votes_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
votes_df.head()

Loading VoteCast events: 100%|██████████| 12425/12425 [00:00<00:00, 543003.60it/s]

Vote cast
	Alpha: Min blockNumber: 9952490 and Max blockNumber: 12126254
	Bravo: Min blockNumber: 12235811 and Max blockNumber: 20555381


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,support,votes,voter,governor,reason,number,timestamp
0,9952490,07076f7d6247f9c5c4703085b8adf30510bf1f96bdc9ba...,e12ef62394fc0264a1f5d37dfde140505a8b16afe35b3b...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,44,35,VoteCast,1,1,24107.64,0xb0325dbe7fa891436e83a094f9f12848c78e449b,alpha,NaN,9952490,2020-04-27 04:54:53
1,9952737,70851da3c89a8c1b30833e57bd52225f0752432dcef84d...,3d6a0c367e462e0a2ea0c09d0d1d96def35aab51857f59...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,99,80,VoteCast,1,1,1.00,0x181fc4a2b228c0e0f996b736f0310cf9b2e34b56,alpha,NaN,9952737,2020-04-27 05:51:06
2,9952767,1d0f2246f485d7862a7b5f97ba19059c69733c7642b4dd...,4ee9556a23676ee9027e9f37db6684cb279de528bc3391...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,93,126,VoteCast,1,1,5000.00,0xe3c9ece23316b6b06142fa0ca915f02c323c6b21,alpha,NaN,9952767,2020-04-27 05:56:47
3,9953038,c8b158cbab11e1ca226bf6ee8a8eeb8337c1771a5f67a3...,83a3068615968e0a86cf52c8c2b2ab917a2013d838f949...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,112,152,VoteCast,1,0,12500.00,0x8703028934b24b47496ad84afd635ea7938487be,alpha,NaN,9953038,2020-04-27 06:53:45
4,9953051,40ad76ab86471ac45a690a2f7cec7be9025940e47699ff...,29ddc362390cdacb28d75864487fff7ff43c6e06d246af...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,135,156,VoteCast,1,1,50000.00,0x1d29756e8f7b091ce6c11a35980de79c7eda5d1f,alpha,NaN,9953051,2020-04-27 06:58:12


In [12]:
print("Number of votess: {}".format(votes_df.shape[0]))
print("Number of unique proposals that received a vote: {}".format(
    votes_df.proposalId.nunique()))
print("Proposals that did not receive a vote: {}".format(
    sorted(set(range(1, votes_df.proposalId.max() + 1)) - set(votes_df.proposalId.unique()))))

Number of votess: 14841
Number of unique proposals that received a vote: 293
Proposals that did not receive a vote: [48, 55, 88, 91, 94, 114, 120, 134, 155, 183, 234, 270, 290, 291]


### Compound Proposal Created


In [13]:
# Convert the ProposalCreated event data to a dataframe
proposal_created_alpha_df = proposal_created_to_dataframe(
    events['compound-governor-alpha']['ProposalCreated'])
proposal_created_bravo_df = proposal_created_to_dataframe(
    events['compound-governor-bravo']['ProposalCreated'])

proposal_created_alpha_df['governor'] = 'alpha'
proposal_created_bravo_df['governor'] = 'bravo'
print("Proposals")
print(
    f"\tAlpha: Min blockNumber: {proposal_created_alpha_df['blockNumber'].min()} and Max blockNumber: {proposal_created_alpha_df['blockNumber'].max()}")
print(
    f"\tBravo: Min blockNumber: {proposal_created_bravo_df['blockNumber'].min()} and Max blockNumber: {proposal_created_bravo_df['blockNumber'].max()}")
proposal_created_df = pd.concat(
    [proposal_created_alpha_df, proposal_created_bravo_df])
proposal_created_df.sort_values(
    by=['blockNumber', 'transactionIndex', 'proposalId'], inplace=True)
del (proposal_created_alpha_df)
del (proposal_created_bravo_df)
print(proposal_created_df.shape)
proposal_created_df['proposal_title'] = proposal_created_df.description.str.split(
    '\n').apply(lambda x: x[0].replace('#', '').strip())
proposal_created_df.drop(columns=['description'], inplace=True)
print("There are {} unique proposals that were created".format(
    proposal_created_df.proposalId.nunique())
)
print("These proposals were created by {} unique proposers".format(
    proposal_created_df.proposer.nunique())
)
proposal_created_df = proposal_created_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
proposal_created_df.head()

Loading ProposalCreated events: 100%|██████████| 265/265 [00:00<00:00, 297110.55it/s]

Proposals
	Alpha: Min blockNumber: 9951904 and Max blockNumber: 12109016
	Bravo: Min blockNumber: 12235671 and Max blockNumber: 20524537
(307, 16)
There are 307 unique proposals that were created
These proposals were created by 45 unique proposers


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,startBlock,endBlock,proposer,targets,values,signatures,governor,proposal_title,number,timestamp
0,9951904,eee9627ef06eb757e7959093c7ba8e3e4e64b466f286df...,9dfcf07b8ffe3c3cf3d4fe3dbc903a93966afb2a8de428...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,38,14,ProposalCreated,1,9951905,9969185,0x8169522c2c57883e8ef80c498aab7820da539806,"0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b,0x3...","0,0","_supportMarket(address),_setPriceOracle(address)",alpha,Add USDT Support,9951904,2020-04-27 02:41:52
1,9956584,9ba70aab4edd5a0ea9f357ef7060ebd6826ddc2fe52b8d...,c70727c7702e8069ece9e2abd4ea53e7e272aac2c23747...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,34,26,ProposalCreated,2,9956585,9973865,0x7e4a8391c728fed9069b2962699ab416628b19fa,0x5d3a536e4d6dbd6114cc1ead35777bab948e3643,0,_setInterestRateModel(address),alpha,Update DAI Interest Rate Model to Better Accom...,9956584,2020-04-27 20:09:05
2,10065392,9c568b241b222a0287c01740c176ad071be4d53b353db2...,1c99453bca042d5cfe5e2a28364149eead7f4d376ecf82...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,121,123,ProposalCreated,3,10065393,10082673,0x54a37d93e57c5da659f508069cf65a381b61e189,"0xf5dce57282a584d2746faf1593d3121fcac444dc,0x3...","0,0","_setReserveFactor(uint256),_setCollateralFacto...",alpha,Changing SAI reserve factor to 100% and collat...,10065392,2020-05-14 16:34:24
3,10117038,f8d137bd9de2e8001828b7b0097f41f9770014cd4d181d...,b35b8f83c48ccbdd0e0cc17caac5db4713e7ea6cedbcbc...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,110,103,ProposalCreated,4,10117039,10134319,0x54a37d93e57c5da659f508069cf65a381b61e189,0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b,0,"_setCollateralFactor(address,uint256)",alpha,Changing SAI collateral factor to 55%,10117038,2020-05-22 17:44:04
4,10179118,9bb530b334db2d1c204f7fa42559535327e7a0f10fc229...,bc5f06c5506765d3c2277b74f2782d6becf8e028331585...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,100,107,ProposalCreated,5,10179119,10196399,0x54a37d93e57c5da659f508069cf65a381b61e189,0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b,0,"_setCollateralFactor(address,uint256)",alpha,Changing SAI collateral factor to 45%,10179118,2020-06-01 09:13:47


### Compound Cancelled Proposals


In [14]:
proposal_cancelled_alpha_df = proposal_cancelled_to_dataframe(
    events['compound-governor-alpha']['ProposalCanceled'])
proposal_cancelled_bravo_df = proposal_cancelled_to_dataframe(
    events['compound-governor-bravo']['ProposalCanceled'])

proposal_cancelled_alpha_df['governor'] = 'alpha'
proposal_cancelled_bravo_df['governor'] = 'bravo'

proposal_cancelled_df = pd.concat([proposal_cancelled_alpha_df, proposal_cancelled_bravo_df]).reset_index(
    drop=True).sort_values(by='proposalId')

del (proposal_cancelled_alpha_df)
del (proposal_cancelled_bravo_df)
# Proposal 270 has two cancel transactions associated with it. We will keep the first one and remove the second one.
proposal_cancelled_df = proposal_cancelled_df.groupby(
    'proposalId').first().reset_index()
print("Number of proposals that were cancelled: {}".format(
    proposal_cancelled_df.proposalId.nunique()))
proposal_cancelled_df = proposal_cancelled_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
proposal_cancelled_df.head()

Loading ProposalCancelled events: 100%|██████████| 29/29 [00:00<00:00, 428291.61it/s]


Number of proposals that were cancelled: 30


,proposalId,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,governor,number,timestamp
0,13,10400372,f716a93007537d87e4f0122c28f12c62da51c3aaa748d5...,3e9597107a7f2b9a76bad3778287d9cce0fb4921b31ac5...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,122,158,ProposalCanceled,alpha,10400372,2020-07-05 16:02:40
1,28,11187005,534dfea338821f21b79e4c329152053a8d10f2af97dfdb...,2e89d4ec70d59abb497bb510607ca212edae4fbbc920fa...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,117,277,ProposalCanceled,alpha,11187005,2020-11-03 22:53:50
2,48,12649400,e6b4d4993b52f3c91c8d211e564e19a3a8616e32adaf39...,beae2b8909303ed2aaa2826f7e7c383e1b9b5e4c3b4dc7...,0xc0da02939e1441f497fd74f78ce7decb17b66529,151,372,ProposalCanceled,bravo,12649400,2021-06-17 03:01:22
3,55,12992395,c4b9fca2bdea3bc6f7e67d1e9b55f477b7439e4675300e...,0a09f130f49e52df8ada8466187e5632fe18b7515cf6fe...,0xc0da02939e1441f497fd74f78ce7decb17b66529,80,110,ProposalCanceled,bravo,12992395,2021-08-09 17:48:39
4,63,13368738,fb6330eb14b12e603f088b3dee2868bf753356b8556399...,151396136573b1345b8052a2af1ebf343ff8205dc7e20c...,0xc0da02939e1441f497fd74f78ce7decb17b66529,197,299,ProposalCanceled,bravo,13368738,2021-10-07 00:46:48


### Compound Executed Proposals


In [15]:
proposal_executed_alpha_df = proposal_executed_to_dataframe(
    events['compound-governor-alpha']['ProposalExecuted'])
proposal_executed_bravo_df = proposal_executed_to_dataframe(
    events['compound-governor-bravo']['ProposalExecuted'])

proposal_executed_alpha_df['governor'] = 'alpha'
proposal_executed_bravo_df['governor'] = 'bravo'

proposal_executed_df = pd.concat([proposal_executed_alpha_df, proposal_executed_bravo_df]).reset_index(
    drop=True).sort_values(by='proposalId')

del (proposal_executed_alpha_df)
del (proposal_executed_bravo_df)
print("Number of proposals that were executed: {}".format(
    proposal_executed_df.proposalId.nunique()))
proposal_executed_df = proposal_executed_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
proposal_executed_df.head()

Loading ProposalExecuted events: 100%|██████████| 202/202 [00:00<00:00, 622062.71it/s]


Number of proposals that were executed: 238


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,governor,number,timestamp
0,9982249,48aeb1532c1cc92103df7d2c00a73c219ca7e143fd41b0...,61edb276e7a3fbee8aff95ad9dc8dda782ca7d3fab0d83...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,20,18,ProposalExecuted,1,alpha,9982249,2020-05-01 19:39:25
1,9987151,4caa5276108eb7800091c100223b31f2f71de2ba218460...,d2edfa0d1c1a505c705ae5493c4fa6e348251336ff132d...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,90,136,ProposalExecuted,2,alpha,9987151,2020-05-02 13:56:43
2,10096844,c92c24d468cb9682b8829fc0e38b44720e1042bc5eb9da...,a23894f5ef1b39ed4e604ffce8f6e520f07d03bc08f78e...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,143,144,ProposalExecuted,3,alpha,10096844,2020-05-19 14:17:08
3,10149088,d6b0f370ac61a926f9bbe19ebb15a34b303c3f1407d17a...,ee7dba9b63e0ebbc3d42ff4e1a38a2d701e40b4178511f...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,118,102,ProposalExecuted,4,alpha,10149088,2020-05-27 17:06:28
4,10211430,2c15915dec2650ffcb796d2bb2075034c931bada321bb0...,b0e43406d2a563e2c03b0b0454a17a25f62f2377aa436f...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,37,29,ProposalExecuted,5,alpha,10211430,2020-06-06 09:49:19


### Compound Queued Proposals


In [16]:
proposal_queued_alpha_df = proposal_queued_to_dataframe(
    events['compound-governor-alpha']['ProposalQueued'])
proposal_queued_bravo_df = proposal_queued_to_dataframe(
    events['compound-governor-bravo']['ProposalQueued'])

proposal_queued_alpha_df['governor'] = 'alpha'
proposal_queued_bravo_df['governor'] = 'bravo'

proposals_queued_df = pd.concat([proposal_queued_alpha_df, proposal_queued_bravo_df]).reset_index(
    drop=True).sort_values(by='proposalId')

del (proposal_queued_alpha_df)
del (proposal_queued_bravo_df)
print("Number of proposals that were queued: {}".format(
    proposals_queued_df.proposalId.nunique()))
proposals_queued_df = proposals_queued_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
proposals_queued_df.head()

Loading ProposalQueued events: 100%|██████████| 210/210 [00:00<00:00, 33180.28it/s]


Number of proposals that were queued: 246


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,eta,governor,number,timestamp
0,9969390,84f1067db81fc00fc1d9abff4742894bc45146a717fe39...,c953d6cdc6a41c492189385b7e64bfb6c51ee8741a9dcd...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,16,24,ProposalQueued,1,2020-05-01 19:37:52,alpha,9969390,2020-04-29 19:37:52
1,9974263,c10b8fe7e13ad447f3733e3d17378e8bb7ba5367331817...,603fd30f7ddb79391fe5f1cdbad9075dd0b49489228eed...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,120,103,ProposalQueued,2,2020-05-02 13:46:48,alpha,9974263,2020-04-30 13:46:48
2,10083622,5d22ef40426138491a6af13eea978cf23e48f10b011310...,416e389e66b3267cf5e38fe6d6677c6bc2d4fdb1c635ed...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,74,103,ProposalQueued,3,2020-05-19 12:35:43,alpha,10083622,2020-05-17 12:35:43
3,10136051,10cf1d1313259a4ace4102d0f56ea9453f4ac3f5a53ef5...,ac9517ea1278fd114709045f3ba334410bbea0133303e3...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,42,301,ProposalQueued,4,2020-05-27 16:36:24,alpha,10136051,2020-05-25 16:36:24
4,10196969,52990be8b4d9a600da301ef1f9ab1fa16c27376074da97...,e531b1f9113dd679412b1b577e7c99c226cf55cfa01b5d...,0xc0da01a04c3f3e0be433606045bb7017a7323e38,58,50,ProposalQueued,5,2020-06-06 03:45:21,alpha,10196969,2020-06-04 03:45:21


### Compound Voting Delay and Period Set


In [17]:
# VotingDelaySet
proposal_voting_delay_df = proposal_voting_delay_to_dataframe(
    events['compound-governor-bravo']['VotingDelaySet'])
proposal_voting_delay_df['governor'] = 'bravo'
proposal_voting_delay_df = proposal_voting_delay_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
display(proposal_voting_delay_df.head())

# VotingPeriodSet
proposal_voting_period_df = voting_period_set_to_dataframe(
    events['compound-governor-bravo']['VotingPeriodSet'])
proposal_voting_period_df['governor'] = 'bravo'
proposal_voting_period_df = proposal_voting_period_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
display(proposal_voting_period_df.head())

Loading ProposalVotingDelaySet events: 100%|██████████| 1/1 [00:00<00:00, 17260.51it/s]


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,oldVotingDelay,newVotingDelay,governor,number,timestamp
0,12266840,bd7e9da57c31931f54c05679a31bb51232b0f2ee3f5fbd...,bfcb44d66b3fd8b4e00f1f3f6902ed47b828fdae8589df...,0xc0da02939e1441f497fd74f78ce7decb17b66529,124,168,VotingDelaySet,1,13140,bravo,12266840,2021-04-18 22:33:00


Loading VotingPeriodSet events: 100%|██████████| 1/1 [00:00<00:00, 30174.85it/s]


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,oldVotingPeriod,newVotingPeriod,governor,number,timestamp
0,12266840,bd7e9da57c31931f54c05679a31bb51232b0f2ee3f5fbd...,bfcb44d66b3fd8b4e00f1f3f6902ed47b828fdae8589df...,0xc0da02939e1441f497fd74f78ce7decb17b66529,124,170,VotingPeriodSet,17280,19710,bravo,12266840,2021-04-18 22:33:00


### Construct datasets for evaluating the proposal voting outcomes


In [18]:
supporters_df = votes_df.groupby(
    'proposalId').support.value_counts().unstack(fill_value=0).rename(columns={0: 'against', 1: 'in_favor', 2: 'abstain'})
supporters_df['n_votes'] = supporters_df.sum(axis=1)
supporters_df['against_percentage'] = 100 * \
    (supporters_df['against'] / supporters_df['n_votes'])
supporters_df['in_favor_percentage'] = 100 * \
    (supporters_df['in_favor'] / supporters_df['n_votes'])
supporters_df['abstain_percentage'] = 100 * \
    (supporters_df['abstain'] / supporters_df['n_votes'])
# supporters_df.add_prefix('supporter_')
supporters_df.columns = ['supporter_' +
                         str(col) for col in supporters_df.columns]
supporters_df.head()

,supporter_against,supporter_in_favor,supporter_abstain,supporter_n_votes,supporter_against_percentage,supporter_in_favor_percentage,supporter_abstain_percentage
proposalId,,,,,,,
1,6,18,0,24,25.000000,75.000000,0.0
2,2,20,0,22,9.090909,90.909091,0.0
3,0,21,0,21,0.000000,100.000000,0.0
4,1,13,0,14,7.142857,92.857143,0.0
5,0,18,0,18,0.000000,100.000000,0.0


In [19]:
# Most weighted voted proposals
votes_weighted_df = votes_df.groupby(
    ['proposalId', 'support']).votes.sum().unstack().fillna(0)
votes_weighted_df.rename(
    columns={0: 'against', 1: 'in_favor', 2: 'abstain'}, inplace=True)
votes_weighted_df['n_votes'] = votes_weighted_df.sum(axis=1)
votes_weighted_df['against_percentage'] = 100 * (votes_weighted_df['against'] /
                                                 votes_weighted_df['n_votes'])
votes_weighted_df['in_favor_percentage'] = 100 * (votes_weighted_df['in_favor'] /
                                                  votes_weighted_df['n_votes'])
votes_weighted_df['abstain_percentage'] = 100 * (votes_weighted_df['abstain'] /
                                                 votes_weighted_df['n_votes'])

votes_weighted_df = votes_weighted_df.merge(
    supporters_df, how='left', left_on='proposalId', right_on='proposalId')

votes_weighted_df = votes_weighted_df.merge(proposal_created_df[['proposalId', 'proposer', 'blockNumber', 'transactionHash', 'proposal_title']], how='outer',
                                            left_on='proposalId', right_on='proposalId')

# Fixing proposal 116 title
votes_weighted_df.loc[votes_weighted_df['proposalId'] == 116,
                      'proposal_title'] = "Initialize Compound III (USDC on Ethereum)"

votes_weighted_df = votes_weighted_df.merge(proposal_created_df.query("proposalId in @proposal_cancelled_df.proposalId")[['proposalId', 'proposer', 'proposal_title']].drop(
    columns=['proposer', 'proposal_title']), on='proposalId', how='outer').fillna(0).sort_values(by='proposalId', ascending=True)[votes_weighted_df.columns]

defeated_proposal_ids = votes_weighted_df.query(
    '(proposalId not in @proposal_executed_df.proposalId) and (proposalId not in @proposal_cancelled_df.proposalId) and (proposalId not in @proposals_queued_df.proposalId)'
).proposalId.unique()

# votes_weighted_df['status'] = 'executed'
votes_weighted_df.loc[votes_weighted_df['proposalId'].isin(
    proposal_executed_df.proposalId), 'status'] = 'executed'
votes_weighted_df.loc[votes_weighted_df['proposalId'].isin(
    proposals_queued_df.proposalId), 'status'] = 'executed'
votes_weighted_df.loc[votes_weighted_df['proposalId'].isin(
    defeated_proposal_ids), 'status'] = 'defeated'
votes_weighted_df.loc[votes_weighted_df['proposalId'].isin(
    proposal_cancelled_df.proposalId), 'status'] = 'canceled'

votes_weighted_df['blockNumber'] = votes_weighted_df['blockNumber'].astype(int)

votes_weighted_df = votes_weighted_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
votes_weighted_df

,proposalId,against,in_favor,abstain,n_votes,against_percentage,in_favor_percentage,abstain_percentage,supporter_against,supporter_in_favor,...,supporter_against_percentage,supporter_in_favor_percentage,supporter_abstain_percentage,proposer,blockNumber,transactionHash,proposal_title,status,number,timestamp
0,1,85580.73,7.979813e+05,0.000000,8.835620e+05,9.685877,90.314123,0.000000,6.0,18.0,...,25.000000,75.000000,0.000000,0x8169522c2c57883e8ef80c498aab7820da539806,9951904,eee9627ef06eb757e7959093c7ba8e3e4e64b466f286df...,Add USDT Support,executed,9951904,2020-04-27 02:41:52
1,2,25001.00,8.982623e+05,0.000000,9.232633e+05,2.707895,97.292105,0.000000,2.0,20.0,...,9.090909,90.909091,0.000000,0x7e4a8391c728fed9069b2962699ab416628b19fa,9956584,9ba70aab4edd5a0ea9f357ef7060ebd6826ddc2fe52b8d...,Update DAI Interest Rate Model to Better Accom...,executed,9956584,2020-04-27 20:09:05
2,3,0.00,7.268129e+05,0.000000,7.268129e+05,0.000000,100.000000,0.000000,0.0,21.0,...,0.000000,100.000000,0.000000,0x54a37d93e57c5da659f508069cf65a381b61e189,10065392,9c568b241b222a0287c01740c176ad071be4d53b353db2...,Changing SAI reserve factor to 100% and collat...,executed,10065392,2020-05-14 16:34:24
3,4,24107.64,4.272289e+05,0.000000,4.513365e+05,5.341389,94.658611,0.000000,1.0,13.0,...,7.142857,92.857143,0.000000,0x54a37d93e57c5da659f508069cf65a381b61e189,10117038,f8d137bd9de2e8001828b7b0097f41f9770014cd4d181d...,Changing SAI collateral factor to 55%,executed,10117038,2020-05-22 17:44:04
4,5,0.00,5.536792e+05,0.000000,5.536792e+05,0.000000,100.000000,0.000000,0.0,18.0,...,0.000000,100.000000,0.000000,0x54a37d93e57c5da659f508069cf65a381b61e189,10179118,9bb530b334db2d1c204f7fa42559535327e7a0f10fc229...,Changing SAI collateral factor to 45%,executed,10179118,2020-06-01 09:13:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,303,0.00,6.221519e+05,0.000000,6.221519e+05,0.000000,100.000000,0.000000,0.0,29.0,...,0.000000,100.000000,0.000000,0x7b3c54e17d618cc94dadfe7671c1e2f50c4ecc33,20474092,f27928cf5e1727ee1a8ffb4ce641ff23af403782420e83...,[Gauntlet] Risk and IO Parameter Recommendatio...,executed,20474092,2024-08-07 03:45:11
303,304,0.00,1.398783e+06,671.812471,1.399455e+06,0.000000,99.951995,0.048005,1.0,36.0,...,2.631579,94.736842,2.631579,0x3fb19771947072629c8eee7995a2ef23b72d4c8a,20500199,eed20a2974640e678b2a20014325f02717aa364c982852...,Add Proposal Guardian to Governor Bravo,executed,20500199,2024-08-10 19:07:11
304,305,0.00,5.622875e+05,0.000000,5.622875e+05,0.000000,100.000000,0.000000,0.0,26.0,...,0.000000,100.000000,0.000000,0xd2a79f263ec55dbc7b724ecc20fc7448d4795a0c,20518659,2d22ae985187e660aa2c7e3906132d25533067e060fbca...,Update Price Feeds on Arbitrum for LSTs,executed,20518659,2024-08-13 08:58:35
305,306,0.00,6.108902e+05,4588.741084,6.154790e+05,0.000000,99.254444,0.745556,0.0,25.0,...,0.000000,96.153846,3.846154,0x683a4f9915d6216f73d6df50151725036bd26c02,20524529,45ebaa9c55302c44ad115e55246bce1c4c1020e5685ba8...,[Gauntlet] Risk Recommendations for LRTs on WE...,executed,20524529,2024-08-14 04:37:59


In [20]:
votes_weighted_df.status.value_counts()

status
executed    241
defeated     36
canceled     30
Name: count, dtype: int64

### Persist Dataset


In [21]:
def persist_dataframe(protocol, df, filename, compression=None):
    os.makedirs(data_dir+protocol+os.sep, exist_ok=True)
    df.to_csv(data_dir+protocol+os.sep + filename,
              index=False, compression=compression)
    print(f"Dataframe persisted to {data_dir + filename}")

In [22]:
persist_dataframe(protocol='compound', df=votes_weighted_df, filename='votes_weighted_df.csv.gz',
                  compression='gzip')
persist_dataframe(protocol='compound', df=proposal_cancelled_df,
                  filename='proposal_cancelled_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=votes_df,
                  filename='votes_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=proposal_created_df,
                  filename='proposal_created_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=proposal_executed_df,
                  filename='proposal_executed_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=proposals_queued_df,
                  filename='proposals_queued_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=proposal_voting_period_df,
                  filename='proposal_voting_period_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=supporters_df,
                  filename='supporters_df.csv.gz', compression='gzip')
persist_dataframe(protocol='compound', df=proposal_voting_delay_df,
                  filename='proposal_voting_delay_df.csv.gz', compression='gzip')

Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/votes_weighted_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_cancelled_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/votes_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_created_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_executed_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposals_queued_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_voting_period_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/supporters_df.csv.gz
Dataframe persisted to /Users/john

## Uniswap Governance


### Uniswap Vote Cast


In [23]:
# Convert the VoteCast event data to dataframe
uniswap_votes_df = vote_cast_to_dataframe(
    events['uniswap-governor-bravo']['VoteCast'])
print("Vote cast")
print(
    f"\tMin blockNumber: {uniswap_votes_df['blockNumber'].min()} and Max blockNumber: {uniswap_votes_df['blockNumber'].max()}")

uniswap_votes_df.sort_values(
    by=['blockNumber', 'transactionIndex', 'proposalId'], inplace=True)

uniswap_votes_df = uniswap_votes_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
uniswap_votes_df.head()

Loading VoteCast events: 100%|██████████| 51580/51580 [00:00<00:00, 544322.84it/s]

Vote cast
	Min blockNumber: 13553145 and Max blockNumber: 20562242


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,support,votes,voter,reason,number,timestamp
0,13553145,d1963da94947c2cd4f10867207fb5b12b08b242f120d8a...,728522ddb2e720b6f5153a023e5cb092373d5f6205791a...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,99,307,VoteCast,9,1,1.000007e+02,0xd724107ae1047b50b543ad4d940dee18e99261f1,,13553145,2021-11-04 23:30:38
1,13553257,7f50b7474f5f003bdb6e6c0cd2310c94f70897fdeb8eac...,56cd5618bc9302b4c02bcc646111a8f762ce52e4a4dc6e...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,145,377,VoteCast,9,1,2.500012e+06,0x9b68c14e936104e9a7a24c712beecdc220002984,,13553257,2021-11-05 00:00:27
2,13554645,862a063d162ee5cd4896c7f05be25cdcf80dfd925b62db...,f1b650360b9ba84d4492a9e47ec3e309758ea74995ed7d...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,204,343,VoteCast,9,1,4.355769e+06,0x458ceec48586a85fcfeb4a179706656ee321730e,,13554645,2021-11-05 05:08:38
3,13556434,faf07fab6405c52eb78d53a2d53c6845679dda8cbbdc76...,67fbaa1ae13110f7aabdd52b325f0b71b550c7cc16a23a...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,314,591,VoteCast,9,1,2.544662e+02,0xf4844a06d4f995c4c03195afcb5aa59dcbb5b4fc,,13556434,2021-11-05 11:52:33
4,13556443,fdb14f0cbeb41fe03b468e102c12b41c32051c22c6b780...,f687493fa7755ea071f117f806b76b3a67f83f522b2df6...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,384,493,VoteCast,9,1,1.054247e+01,0xe87be8e831c019b1113f1ea5f8ffd9ba65828bf2,,13556443,2021-11-05 11:54:31


In [24]:
print("Number of votess: {}".format(uniswap_votes_df.shape[0]))
print("Number of unique proposals that received a vote: {}".format(
    uniswap_votes_df.proposalId.nunique()))
print("Proposals that did not receive a vote: {}".format(
    sorted(set(range(1, uniswap_votes_df.proposalId.max() + 1)) - set(uniswap_votes_df.proposalId.unique()))))

Number of votess: 51580
Number of unique proposals that received a vote: 50
Proposals that did not receive a vote: [1, 2, 3, 4, 5, 6, 7, 8, 12, 15, 21, 26, 27, 35, 44, 48, 53]


### Uniswap Proposal Created


In [25]:
# Convert the ProposalCreated event data to a dataframe

uniswap_proposal_created_df = proposal_created_to_dataframe(
    events['uniswap-governor-bravo']['ProposalCreated'])

print("Proposals")
print(
    f"Min blockNumber: {uniswap_proposal_created_df['blockNumber'].min()} and Max blockNumber: {uniswap_proposal_created_df['blockNumber'].max()}")

uniswap_proposal_created_df.sort_values(
    by=['blockNumber', 'transactionIndex', 'proposalId'], inplace=True)

print(uniswap_proposal_created_df.shape)
uniswap_proposal_created_df['proposal_title'] = uniswap_proposal_created_df.description.str.split(
    '\n').apply(lambda x: x[0].replace('#', '').strip())
uniswap_proposal_created_df.drop(columns=['description'], inplace=True)
print("There are {} unique proposals that were created".format(
    uniswap_proposal_created_df.proposalId.nunique())
)
print("These proposals were created by {} unique proposers".format(
    uniswap_proposal_created_df.proposer.nunique())
)
uniswap_proposal_created_df = uniswap_proposal_created_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
uniswap_proposal_created_df.head()

Loading ProposalCreated events: 100%|██████████| 67/67 [00:00<00:00, 252714.36it/s]

Proposals
Min blockNumber: 13129516 and Max blockNumber: 20529031
(67, 15)
There are 67 unique proposals that were created
These proposals were created by 31 unique proposers


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,startBlock,endBlock,proposer,targets,values,signatures,proposal_title,number,timestamp
0,13129516,52371bf3cc7dc7169203e70e2e914c31408021d38c6b44...,ea3efc1877926b231d9e454601f944ad83a3e6a8f00135...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,36,38,ProposalCreated,1,13142656,13182976,0x81a7f10003d5ca866da96299ff1d3c673afcc138,0x0000000000000000000000000000000000000000,0,,"""""",13129516,2021-08-30 22:16:12
1,13129539,847596f77ed4c5ddeb725494adeade3d9e5ae6b64cc568...,0de6aa12284c443bb98a3f96ad39b245a1b14ecb1cfe1d...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,145,163,ProposalCreated,2,13142679,13182999,0x23122238253343ea36088ee1f19951904594506f,0x0000000000000000000000000000000000000000,0,,"""""",13129539,2021-08-30 22:20:33
2,13129552,95670229440b845b62716907046431a00f98a1bb56d1eb...,06e13287998965af909f8364ebb1639e2f808df33a8ffc...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,72,74,ProposalCreated,3,13142692,13183012,0x4cabb1dd23515747c483a95ef2b60731699ceaea,0x0000000000000000000000000000000000000000,0,,"""""",13129552,2021-08-30 22:23:48
3,13129576,772be6ec0533a82a7d24baa32fb9af39c068a293adc733...,484b2c66727aadda4a5084888bc5068f1959a10802184f...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,353,428,ProposalCreated,4,13142716,13183036,0xb4428b8f878d60834bb2126c56735d0827a7ccda,0x0000000000000000000000000000000000000000,0,,"""""",13129576,2021-08-30 22:27:58
4,13129600,b0a51195f51af03d0ac19098d66a40caf35b1e1be55561...,3b1969cf091e5f7ac3c106a2b6623efdb2f75bc4930a36...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,392,384,ProposalCreated,5,13142740,13183060,0xce3c6ee6d78b59b2aa66050d2d31d3223c850574,0x0000000000000000000000000000000000000000,0,,"""""",13129600,2021-08-30 22:32:20


### Uniswap Cancelled Proposals


In [26]:
uniswap_proposal_cancelled_df = proposal_cancelled_to_dataframe(
    events['uniswap-governor-bravo']['ProposalCanceled'])

uniswap_proposal_cancelled_df = uniswap_proposal_cancelled_df.groupby(
    'proposalId').first().reset_index()
print("Number of proposals that were cancelled: {}".format(
    uniswap_proposal_cancelled_df.proposalId.nunique()))
uniswap_proposal_cancelled_df = uniswap_proposal_cancelled_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
uniswap_proposal_cancelled_df.head()

Loading ProposalCancelled events: 100%|██████████| 22/22 [00:00<00:00, 415651.75it/s]


Number of proposals that were cancelled: 14


,proposalId,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,number,timestamp
0,12,14404501,01edddc2b5f01138d3fdc81feb9c9c7ad799ce0b8675cb...,997ef0adcca09e70b1d90f3c01a0a9ba346c61eae4163a...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,67,75,ProposalCanceled,14404501,2022-03-17 14:15:23
1,15,14618340,3865e983b853559771fa5304ec150be1578587bb36795b...,2ddddc5164b2249a7e6407bd9424181f83fc68804f38c6...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,158,291,ProposalCanceled,14618340,2022-04-19 22:47:40
2,17,14756434,48a3fa2dcf210c0f76089d2f1cd1ac1e56d01e871b4bff...,a9adf79c55d62222b588c7a3a25a736603ab13b0fac63d...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,195,193,ProposalCanceled,14756434,2022-05-11 18:27:01
3,18,14750247,a8ad4782b97ad96e34c669b076f0b5a88fb6f7f4963776...,2d9317408e146301ea69487fc5627f80df02c96715d2b4...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,172,140,ProposalCanceled,14750247,2022-05-10 18:58:44
4,21,14968829,e5610b0dcff58ffc066444e6a9195814984e33259ba78d...,ec67a92ec8747dcde429bfb31cc8dd3a118a39737c515b...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,236,285,ProposalCanceled,14968829,2022-06-15 18:00:04


### Uniswap Executed Proposals


In [27]:
uniswap_proposal_executed_df = proposal_executed_to_dataframe(
    events['uniswap-governor-bravo']['ProposalExecuted'])

print("Number of proposals that were executed: {}".format(
    uniswap_proposal_executed_df.proposalId.nunique()))
uniswap_proposal_executed_df = uniswap_proposal_executed_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
uniswap_proposal_executed_df.head()

Loading ProposalExecuted events: 100%|██████████| 43/43 [00:00<00:00, 477129.82it/s]


Number of proposals that were executed: 43


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,number,timestamp
0,13604706,5c84f89a67237db7500538b81af61ebd827c081302dd73...,2e038c3226f41890c9f2942ce10a76ad5cb723b7fb9f8d...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,244,354,ProposalExecuted,9,13604706,2021-11-13 01:15:41
1,13861316,4f4eac8d27ed3fe63e605d397a59eb31b4880d9ea58839...,34cec132090712cf29614b2ed4c67eb6bea47e0803fec5...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,378,458,ProposalExecuted,10,13861316,2021-12-23 12:07:44
2,14372520,aea63dc6b8d71c3a5559b43df7a1210c2f0847d2670d17...,acc9ef2ab761458ee9be1c9f781ba3873d0367738ed712...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,100,200,ProposalExecuted,11,14372520,2022-03-12 14:41:08
3,14501529,1b68b657b72b5895ee359eafbda696601e157469f10641...,b2b8a1ca5703cf50261fbcf1d831f60f5a47fc13cdf505...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,266,389,ProposalExecuted,13,14501529,2022-04-01 16:42:29
4,14567223,15dec50eed28ab14b2fdf5429e29d7acc32e68d23594ed...,96b18ff79b32f491a52e4dc0d7108a670b886791dbd7e1...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,149,312,ProposalExecuted,14,14567223,2022-04-11 23:08:57


### Uniswap Queued Proposals


In [28]:
uniswap_proposal_queued_df = proposal_queued_to_dataframe(
    events['uniswap-governor-bravo']['ProposalQueued'])

print("Number of proposals that were queued: {}".format(
    uniswap_proposal_queued_df.proposalId.nunique()))
uniswap_proposal_queued_df = uniswap_proposal_queued_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
uniswap_proposal_queued_df.head()

Loading ProposalQueued events: 100%|██████████| 43/43 [00:00<00:00, 20986.16it/s]


Number of proposals that were queued: 43


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,proposalId,eta,number,timestamp
0,13591709,e91e2bcb8edfdf815606cf7d9470d6030286cd470e26e5...,2ff5020685f087f99d402ac6f94acbaa481697fb31840b...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,311,539,ProposalQueued,9,2021-11-13 00:26:35,13591709,2021-11-11 00:26:35
1,13827677,a66629d2d1e9c568d22eda96911bddaad84c517f9fab7c...,42496c6802bceea37dde81fc5be19906101e4c676bc8c8...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,296,565,ProposalQueued,10,2021-12-20 07:13:01,13827677,2021-12-18 07:13:01
2,14355132,86a450c72a343334792b26342cbf1a6d2ea3dbb99e9160...,4f9ce9c3d7730b3f4a55921c24bf372e2496c17e98d4e1...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,167,278,ProposalQueued,11,2022-03-11 21:55:43,14355132,2022-03-09 21:55:43
3,14476169,bc84001a6dc11cad9369cb9cb8a9cc776433a64082baad...,ade81bbf2981006ac4ff7d6cf0c12d75dbcf4fc27fe361...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,140,148,ProposalQueued,13,2022-03-30 18:07:44,14476169,2022-03-28 18:07:44
4,14553555,b8711af162afd9aa83813c256452f5e603c484119907f3...,643e62d541636d284f759675e8ee322bfcd2ad8393ae33...,0x408ed6354d4973f66138c91495f2f2fcbd8724c3,262,297,ProposalQueued,14,2022-04-11 19:57:48,14553555,2022-04-09 19:57:48


### Construct datasets for evaluating the proposal voting outcomes


In [29]:
uniswap_supporters_df = uniswap_votes_df.groupby(
    'proposalId').support.value_counts().unstack(fill_value=0).rename(columns={0: 'against', 1: 'in_favor', 2: 'abstain'})
uniswap_supporters_df['n_votes'] = uniswap_supporters_df.sum(axis=1)
uniswap_supporters_df['against_percentage'] = 100 * \
    (uniswap_supporters_df['against'] / uniswap_supporters_df['n_votes'])
uniswap_supporters_df['in_favor_percentage'] = 100 * \
    (uniswap_supporters_df['in_favor'] / uniswap_supporters_df['n_votes'])
uniswap_supporters_df['abstain_percentage'] = 100 * \
    (uniswap_supporters_df['abstain'] / uniswap_supporters_df['n_votes'])

uniswap_supporters_df.columns = ['supporter_' +
                                 str(col) for col in uniswap_supporters_df.columns]
uniswap_supporters_df.head()

,supporter_against,supporter_in_favor,supporter_abstain,supporter_n_votes,supporter_against_percentage,supporter_in_favor_percentage,supporter_abstain_percentage
proposalId,,,,,,,
9,0,41,0,41,0.000000,100.000000,0.000000
10,4,71,0,75,5.333333,94.666667,0.000000
11,16,1949,0,1965,0.814249,99.185751,0.000000
13,9,1121,1,1131,0.795756,99.115827,0.088417
14,0,311,0,311,0.000000,100.000000,0.000000


In [30]:
# Most weighted voted proposals
uniswap_votes_weighted_df = uniswap_votes_df.groupby(
    ['proposalId', 'support']).votes.sum().unstack().fillna(0)
uniswap_votes_weighted_df.rename(
    columns={0: 'against', 1: 'in_favor', 2: 'abstain'}, inplace=True)
uniswap_votes_weighted_df['n_votes'] = uniswap_votes_weighted_df.sum(axis=1)
uniswap_votes_weighted_df['against_percentage'] = 100 * (uniswap_votes_weighted_df['against'] /
                                                         uniswap_votes_weighted_df['n_votes'])
uniswap_votes_weighted_df['in_favor_percentage'] = 100 * (uniswap_votes_weighted_df['in_favor'] /
                                                          uniswap_votes_weighted_df['n_votes'])
uniswap_votes_weighted_df['abstain_percentage'] = 100 * (uniswap_votes_weighted_df['abstain'] /
                                                         uniswap_votes_weighted_df['n_votes'])

uniswap_votes_weighted_df = uniswap_votes_weighted_df.merge(
    uniswap_supporters_df, how='left', left_on='proposalId', right_on='proposalId')

uniswap_votes_weighted_df = uniswap_votes_weighted_df.merge(uniswap_proposal_created_df[['proposalId', 'proposer', 'blockNumber', 'transactionHash', 'proposal_title']], how='outer',
                                                            left_on='proposalId', right_on='proposalId')

uniswap_votes_weighted_df = uniswap_votes_weighted_df.merge(uniswap_proposal_created_df.query("proposalId in @uniswap_proposal_cancelled_df.proposalId")[['proposalId', 'proposer', 'proposal_title']].drop(
    columns=['proposer', 'proposal_title']), on='proposalId', how='outer').fillna(0).sort_values(by='proposalId', ascending=True)[uniswap_votes_weighted_df.columns]


uniswap_defeated_proposal_ids = uniswap_votes_weighted_df.query(
    '(proposalId not in @uniswap_proposal_executed_df.proposalId) and (proposalId not in @uniswap_proposal_cancelled_df.proposalId) and (proposalId not in @uniswap_proposal_queued_df.proposalId)'
).proposalId.unique()

# uniswap_votes_weighted_df['status'] = 'executed'
uniswap_votes_weighted_df.loc[uniswap_votes_weighted_df['proposalId'].isin(
    uniswap_proposal_executed_df.proposalId), 'status'] = 'executed'
uniswap_votes_weighted_df.loc[uniswap_votes_weighted_df['proposalId'].isin(
    uniswap_proposal_queued_df.proposalId), 'status'] = 'executed'
uniswap_votes_weighted_df.loc[uniswap_votes_weighted_df['proposalId'].isin(
    uniswap_defeated_proposal_ids), 'status'] = 'defeated'
uniswap_votes_weighted_df.loc[uniswap_votes_weighted_df['proposalId'].isin(
    uniswap_proposal_cancelled_df.proposalId), 'status'] = 'canceled'

uniswap_votes_weighted_df['blockNumber'] = uniswap_votes_weighted_df['blockNumber'].astype(
    int)

uniswap_votes_weighted_df = uniswap_votes_weighted_df.merge(
    blocks_df, left_on='blockNumber', right_on='number', how='left')
uniswap_votes_weighted_df

,proposalId,against,in_favor,abstain,n_votes,against_percentage,in_favor_percentage,abstain_percentage,supporter_against,supporter_in_favor,...,supporter_against_percentage,supporter_in_favor_percentage,supporter_abstain_percentage,proposer,blockNumber,transactionHash,proposal_title,status,number,timestamp
0,1,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0x81a7f10003d5ca866da96299ff1d3c673afcc138,13129516,52371bf3cc7dc7169203e70e2e914c31408021d38c6b44...,"""""",defeated,13129516,2021-08-30 22:16:12
1,2,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0x23122238253343ea36088ee1f19951904594506f,13129539,847596f77ed4c5ddeb725494adeade3d9e5ae6b64cc568...,"""""",defeated,13129539,2021-08-30 22:20:33
2,3,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0x4cabb1dd23515747c483a95ef2b60731699ceaea,13129552,95670229440b845b62716907046431a00f98a1bb56d1eb...,"""""",defeated,13129552,2021-08-30 22:23:48
3,4,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0xb4428b8f878d60834bb2126c56735d0827a7ccda,13129576,772be6ec0533a82a7d24baa32fb9af39c068a293adc733...,"""""",defeated,13129576,2021-08-30 22:27:58
4,5,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0xce3c6ee6d78b59b2aa66050d2d31d3223c850574,13129600,b0a51195f51af03d0ac19098d66a40caf35b1e1be55561...,"""""",defeated,13129600,2021-08-30 22:32:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,63,1538.503959,4.946926e+07,9.169885e+05,5.038779e+07,0.003053,98.177084,1.819863,19.0,616.0,...,2.964119,96.099844,0.936037,0xa6e8772af29b29b9202a073f8e36f447689beef6,19918795,578ef4a4d1e544e0238e361fcd17adf9fe948c69884f41...,DeFi Education Fund,executed,19918795,2024-05-21 14:11:35
63,64,177.254395,4.384595e+07,1.000000e+02,4.384623e+07,0.000404,99.999368,0.000228,25.0,158.0,...,13.586957,85.869565,0.543478,0xecc2a9240268bc7a26386ecb49e1befca2706ac9,19947066,c12edd24e2c4939aaae797a8e665ce19ed6e822976ae9e...,Uniswap Delegate Reward- 3 Months Cycle 1,executed,19947066,2024-05-25 13:03:11
64,65,38.974484,4.498072e+07,7.751792e+06,5.273255e+07,0.000074,85.299722,14.700204,36.0,244.0,...,12.720848,86.219081,1.060071,0x3fb19771947072629c8eee7995a2ef23b72d4c8a,20150689,098db9b1801a542d45b521684c1fcac68ec26eb3112794...,Uniswap Arbitrum LTIPP Matching,executed,20150689,2024-06-23 00:02:35
65,66,256839.322986,4.960010e+07,0.000000e+00,4.985694e+07,0.515153,99.484847,0.000000,87.0,463.0,...,15.818182,84.181818,0.000000,0x3fb19771947072629c8eee7995a2ef23b72d4c8a,20478699,e8d88bfc8561b13ec4633c9f7300fee1079898db748d48...,Onboarding Package for Gnosis Chain,executed,20478699,2024-08-07 19:10:35


In [31]:
uniswap_votes_weighted_df.status.value_counts()

status
executed    43
canceled    14
defeated    10
Name: count, dtype: int64

In [32]:
def persist_dataframe(protocol, df, filename, compression=None):
    os.makedirs(data_dir+protocol+os.sep, exist_ok=True)
    df.to_csv(data_dir+protocol+os.sep + filename.lower(),
              index=False, compression=compression)
    print(f"Dataframe persisted to {data_dir + filename}")

In [33]:
persist_dataframe(protocol='uniswap', df=uniswap_votes_weighted_df, filename='votes_weighted_df.csv.gz',
                  compression='gzip')
persist_dataframe(protocol='uniswap', df=uniswap_proposal_cancelled_df,
                  filename='proposal_cancelled_df.csv.gz', compression='gzip')
persist_dataframe(protocol='uniswap', df=uniswap_votes_df,
                  filename='votes_df.csv.gz', compression='gzip')
persist_dataframe(protocol='uniswap', df=uniswap_proposal_created_df,
                  filename='proposal_created_df.csv.gz', compression='gzip')
persist_dataframe(protocol='uniswap', df=uniswap_proposal_executed_df,
                  filename='proposal_executed_df.csv.gz', compression='gzip')
persist_dataframe(protocol='uniswap', df=uniswap_proposal_queued_df,
                  filename='proposals_queued_df.csv.gz', compression='gzip')
persist_dataframe(protocol='uniswap', df=uniswap_supporters_df,
                  filename='supporters_df.csv.gz', compression='gzip')

Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/votes_weighted_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_cancelled_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/votes_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_created_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposal_executed_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/proposals_queued_df.csv.gz
Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/supporters_df.csv.gz


# Token Dataset Parser


## Compound COMP Token Data Parser


In [34]:
contract_names = ['compound-token', 'uniswap-token']

In [35]:
# Load all contract events from contract
events = dict()
for contract_name in contract_names:
    print(f"Loading events for {contract_name}")
    events[contract_name] = load_dataset(contract_name)

Loading events for compound-token
Loading events for uniswap-token


In [36]:
events[contract_name].keys()

dict_keys(['Approval', 'DelegateChanged', 'DelegateVotesChanged', 'MinterChanged', 'Transfer'])

In [37]:
# Convert the Approval event data to a dataframe
events_parsed = dict()
for contract_name in contract_names:
    display(f"Processing {contract_name}")
    events_parsed[contract_name] = dict()
    for event_name in events[contract_name].keys():
        func = get_func(event_name)
        events_parsed[contract_name][event_name] = func(
            events[contract_name][event_name])
        print(">>> {}: There are {} unique {} events.".format(
            contract_name, events_parsed[contract_name][event_name].shape[0], event_name))
        display(events_parsed[contract_name][event_name].head())

'Processing compound-token'

Loading Approval events: 100%|██████████| 256032/256032 [00:00<00:00, 659660.22it/s]


>>> compound-token: There are 256032 unique Approval events.


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,owner,spender,amount
0,10272036,da7763425998c2773a9df0d57e8e14a9a1e4b6f688a8dd...,d48cfaa1c47f1c29892ecab2fa7ed192db6d805247a2ce...,0xc00e94cb662c3520282e6f5717214004a7f26888,33,154,Approval,0x807a74f40f8f41c9d46b6543951c615af66bb77a,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
1,10272063,ce905656bd4c4e31143b8889183eba512f781062f66425...,2288f7c1b4b6696ac4ac1846871f28838978e154bdef50...,0xc00e94cb662c3520282e6f5717214004a7f26888,101,9,Approval,0xd8d7e51caae47e7b23ca21bda30cf6f3e9e006f3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
2,10272074,b951900f79f01afaf625f3c8a58d1320f7accda8c66f27...,040f8a2cbd78bd35fd37a220b78bfb2ac974d028ab5a7f...,0xc00e94cb662c3520282e6f5717214004a7f26888,0,0,Approval,0xc547a1502da650fab2c71f6d440d585788facade,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
3,10272074,1e8905eace4c8318433566a31106f1ceade0250167693f...,040f8a2cbd78bd35fd37a220b78bfb2ac974d028ab5a7f...,0xc00e94cb662c3520282e6f5717214004a7f26888,20,26,Approval,0x51f4f6bb08338bf315d895acec4695b8ab12ba30,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
4,10272081,29dcd6ea23595e9e6d095b4f216dc37c6ff7729b39a2c4...,2d47767b794339ae589b76c7ee4fb382f641f7a714a118...,0xc00e94cb662c3520282e6f5717214004a7f26888,119,175,Approval,0x4b0da7f1868a583893a76ab2ab3ae89c382ff3ff,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10


Loading DelegateChanged events: 100%|██████████| 15244/15244 [00:00<00:00, 753381.37it/s]

...

>>> uniswap-token: There are 2 unique MinterChanged events.


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,minter,newMinter
0,10861674,4b37d2f343608457ca3322accdab2811c707acf3eb07a4...,32e4dd857b5b7e756551a00271e44b61dbda0a91db951c...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,9,24,MinterChanged,0x0000000000000000000000000000000000000000,0x41653c7d61609d856f29355e404f310ec4142cfb
1,10875207,69e088e94c3933a3706360b41fbc4df0947d457fde5bba...,540a4ab4970bdfb506b35b4cbf2e80af4e9c1dcd01b0b8...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0,0,MinterChanged,0x41653c7d61609d856f29355e404f310ec4142cfb,0x1a9c8182c09f50c8318d769245bea52c32be35bc


Loading Transfer events: 100%|██████████| 4765566/4765566 [00:06<00:00, 772976.84it/s]


>>> uniswap-token: There are 4765566 unique Transfer events.


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount
0,10861674,4b37d2f343608457ca3322accdab2811c707acf3eb07a4...,32e4dd857b5b7e756551a00271e44b61dbda0a91db951c...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,9,23,Transfer,0x0000000000000000000000000000000000000000,0x41653c7d61609d856f29355e404f310ec4142cfb,1.000000e+09
1,10861766,0169d8b766b66447e7f869ea46980555bb7f377941d93d...,51a1a8789536990bcca505f514e03d44af25022decb582...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,65,202,Transfer,0x41653c7d61609d856f29355e404f310ec4142cfb,0xe5737257d9406019768167c26f5c6123864cec1e,2.000000e+00
2,10861777,f8edbd74ef0ab3da0b0b2303aab50a39b2597bb3ccc864...,e449f5bea40fb94c014fc78977a3893e4ed1441c6d4e0f...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,76,158,Transfer,0x41653c7d61609d856f29355e404f310ec4142cfb,0xe5737257d9406019768167c26f5c6123864cec1e,8.000000e+00
3,10862390,62f4b047a954be261b9e76c119516647fd2edde4265111...,b4fcd45357272fe8e2a7c5564505cc658abf4c3bc70170...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,64,218,Transfer,0xe5737257d9406019768167c26f5c6123864cec1e,0x1c8bca22fa5f57c3e289ed469bcae62baf63edf7,2.000000e+00
4,10862404,2ca0f926e58a9a3f195702726b52506547c9c9b03b92c3...,e2115ea5048ff28af165c3637c4793504c29a434728e02...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,55,75,Transfer,0xe5737257d9406019768167c26f5c6123864cec1e,0x557847aa724ea004041522ee550e1ae14ced6d7e,2.000000e+00


### Persist Dataset


In [38]:
for contract_name in contract_names:
    display(f"Processing {contract_name}")
    for event_name in events_parsed[contract_name].keys():
        print('\t', event_name)
        display(events_parsed[contract_name][event_name].head())
        persist_dataframe(protocol=contract_name.split('-')[0], df=events_parsed[contract_name][event_name],
                          filename=event_name+'_df.csv.gz', compression='gzip')

'Processing compound-token'

	 Approval


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,owner,spender,amount
0,10272036,da7763425998c2773a9df0d57e8e14a9a1e4b6f688a8dd...,d48cfaa1c47f1c29892ecab2fa7ed192db6d805247a2ce...,0xc00e94cb662c3520282e6f5717214004a7f26888,33,154,Approval,0x807a74f40f8f41c9d46b6543951c615af66bb77a,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
1,10272063,ce905656bd4c4e31143b8889183eba512f781062f66425...,2288f7c1b4b6696ac4ac1846871f28838978e154bdef50...,0xc00e94cb662c3520282e6f5717214004a7f26888,101,9,Approval,0xd8d7e51caae47e7b23ca21bda30cf6f3e9e006f3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
2,10272074,b951900f79f01afaf625f3c8a58d1320f7accda8c66f27...,040f8a2cbd78bd35fd37a220b78bfb2ac974d028ab5a7f...,0xc00e94cb662c3520282e6f5717214004a7f26888,0,0,Approval,0xc547a1502da650fab2c71f6d440d585788facade,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
3,10272074,1e8905eace4c8318433566a31106f1ceade0250167693f...,040f8a2cbd78bd35fd37a220b78bfb2ac974d028ab5a7f...,0xc00e94cb662c3520282e6f5717214004a7f26888,20,26,Approval,0x51f4f6bb08338bf315d895acec4695b8ab12ba30,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10
4,10272081,29dcd6ea23595e9e6d095b4f216dc37c6ff7729b39a2c4...,2d47767b794339ae589b76c7ee4fb382f641f7a714a118...,0xc00e94cb662c3520282e6f5717214004a7f26888,119,175,Approval,0x4b0da7f1868a583893a76ab2ab3ae89c382ff3ff,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7.922816e+10


Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/Approval_df.csv.gz
	 DelegateChanged


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,delegator,fromDelegate,toDelegate
0,9646915,f6aad091cecb553ce7904104b5e04c009db5fa43e3046b...,05964a2e3105233be041912bb518b2c6a584ee527827b0...,0xc00e94cb662c3520282e6f5717214004a7f26888,18,23,DelegateChanged,0x49964dca5c033b1ac8a58d550576c69b91d55e75,0x0000000000000000000000000000000000000000,0x2dbd55a346c1db79062f1b07e8beb963534b9d7e
1,9658580,005aec8e5bf2db204bf341bfef5c5c7da4809a8932fc40...,88ccbc8a6f92a0499f626b392dbb6b591dfadca50060bd...,0xc00e94cb662c3520282e6f5717214004a7f26888,8,5,DelegateChanged,0x49964dca5c033b1ac8a58d550576c69b91d55e75,0x2dbd55a346c1db79062f1b07e8beb963534b9d7e,0x73905a4d38b749be7d5674418ef36b43fa5ec280
2,9686030,528ccc9baf339a4b0cb2645f4dd2344dc18a6a05bc260d...,3398225f53e2e75569081b220ca133c16f30fceb5c8afb...,0xc00e94cb662c3520282e6f5717214004a7f26888,130,112,DelegateChanged,0xb59c8729d3627842a6c2d4b0c2c605ab95794596,0x0000000000000000000000000000000000000000,0xe12283144353f56b7f7f37cd30a8e4a6b7827efc
3,9697764,87e3b3d3a3c9925881bff8aa47134ef0e1c17724806937...,da988fda3d3a7c15033082958b13cf36d7e52435eb7d00...,0xc00e94cb662c3520282e6f5717214004a7f26888,15,12,DelegateChanged,0xb59c8729d3627842a6c2d4b0c2c605ab95794596,0xe12283144353f56b7f7f37cd30a8e4a6b7827efc,0x2ad9ea1e677949a536a270cec812d6e868c88108
4,9697893,c7dd3987db695e9a56bd5abfc948e7f563217c93cf343c...,89acad73c8ef99dcd75a924a4584d957a83462c511da77...,0xc00e94cb662c3520282e6f5717214004a7f26888,131,136,DelegateChanged,0x046231a12d30248bad3322af74cea9c325627d32,0x0000000000000000000000000000000000000000,0x046231a12d30248bad3322af74cea9c325627d32


...

Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/DelegateVotesChanged_df.csv.gz
	 MinterChanged


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,minter,newMinter
0,10861674,4b37d2f343608457ca3322accdab2811c707acf3eb07a4...,32e4dd857b5b7e756551a00271e44b61dbda0a91db951c...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,9,24,MinterChanged,0x0000000000000000000000000000000000000000,0x41653c7d61609d856f29355e404f310ec4142cfb
1,10875207,69e088e94c3933a3706360b41fbc4df0947d457fde5bba...,540a4ab4970bdfb506b35b4cbf2e80af4e9c1dcd01b0b8...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0,0,MinterChanged,0x41653c7d61609d856f29355e404f310ec4142cfb,0x1a9c8182c09f50c8318d769245bea52c32be35bc


Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/MinterChanged_df.csv.gz
	 Transfer


,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount
0,10861674,4b37d2f343608457ca3322accdab2811c707acf3eb07a4...,32e4dd857b5b7e756551a00271e44b61dbda0a91db951c...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,9,23,Transfer,0x0000000000000000000000000000000000000000,0x41653c7d61609d856f29355e404f310ec4142cfb,1.000000e+09
1,10861766,0169d8b766b66447e7f869ea46980555bb7f377941d93d...,51a1a8789536990bcca505f514e03d44af25022decb582...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,65,202,Transfer,0x41653c7d61609d856f29355e404f310ec4142cfb,0xe5737257d9406019768167c26f5c6123864cec1e,2.000000e+00
2,10861777,f8edbd74ef0ab3da0b0b2303aab50a39b2597bb3ccc864...,e449f5bea40fb94c014fc78977a3893e4ed1441c6d4e0f...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,76,158,Transfer,0x41653c7d61609d856f29355e404f310ec4142cfb,0xe5737257d9406019768167c26f5c6123864cec1e,8.000000e+00
3,10862390,62f4b047a954be261b9e76c119516647fd2edde4265111...,b4fcd45357272fe8e2a7c5564505cc658abf4c3bc70170...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,64,218,Transfer,0xe5737257d9406019768167c26f5c6123864cec1e,0x1c8bca22fa5f57c3e289ed469bcae62baf63edf7,2.000000e+00
4,10862404,2ca0f926e58a9a3f195702726b52506547c9c9b03b92c3...,e2115ea5048ff28af165c3637c4793504c29a434728e02...,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,55,75,Transfer,0xe5737257d9406019768167c26f5c6123864cec1e,0x557847aa724ea004041522ee550e1ae14ced6d7e,2.000000e+00


Dataframe persisted to /Users/johnnatanmessias/research-projects/research-proj-governance/data/Transfer_df.csv.gz


## Transactions Related to Vote Cost


In [39]:
file_dir = os.path.join(data_dir, 'transactions_of_votes.pkl.gz')
with gzip.open(file_dir, 'rb') as f:
    txs_data = pickle.load(f)

In [40]:
transactions_df = pd.DataFrame(map(lambda tx: {
    'tx_hash': tx['tx']['hash'].hex().lower(),
    'tx_block_number': tx['tx']['blockNumber'],
    'tx_effective_gas_price': tx['receipt']['effectiveGasPrice'],
    'tx_gas_used': tx['receipt']['gasUsed'],
    'tx_fee_eth': (tx['receipt']['effectiveGasPrice'] * tx['receipt']['gasUsed'])/1e18,
}, txs_data))
print("The dataset contains {} transactions".format(transactions_df.shape[0]))
transactions_df = transactions_df.merge(
    blocks_df, left_on='tx_block_number', right_on='number', how='left').drop(columns=['number'])

file_dir = os.path.join(data_dir, 'transactions_of_votes.csv.gz')
transactions_df.to_csv(file_dir, index=False, compression='gzip')
transactions_df.head()

The dataset contains 65902 transactions


,tx_hash,tx_block_number,tx_effective_gas_price,tx_gas_used,tx_fee_eth,timestamp
0,43cf2f48d2d77816123fb95bc8aa7394cd6d06b7ee219f...,15703859,7546215611,79856,0.000603,2022-10-08 14:05:59
1,c8b44a55e5d9a2593bc110b63362c3d84e5a1fee041575...,17996706,12168780679,79856,0.000972,2023-08-26 05:16:47
2,9dfeb0296e685b3a15fa407715583732045bb9a81e6e3d...,19507380,17965880351,79984,0.001437,2024-03-24 22:32:23
3,dad8a397ed3725c5ae2febd0f9f128c6ab01f17435718a...,10226261,30500000000,63134,0.001926,2020-06-08 16:45:48
4,f69da2f52f4e3b6f2797ac40a9d1edf21d0ef9802cbd4d...,13225977,68000000000,80554,0.005478,2021-09-14 20:23:28


# Accounts Labels Parser


In [41]:
labels_dir = os.path.join(data_dir, "labels")
filenames = [filename for filename in os.listdir(
    labels_dir) if filename.endswith(".csv.gz")]
filenames

['tally_uniswap_account_labels.csv.gz',
 'etherscan_account_labels.csv.gz',
 'tally_compound_account_labels.csv.gz',
 'sybil_list_account_labels.csv.gz']

In [42]:
dfs = {}
for filename in filenames:
    file_dir = os.path.join(labels_dir, filename)
    df = pd.read_csv(file_dir, compression='gzip')
    filename_clean = filename.replace('.csv.gz', '')
    dfs[filename_clean] = df
dfs.keys()

dict_keys(['tally_uniswap_account_labels', 'etherscan_account_labels', 'tally_compound_account_labels', 'sybil_list_account_labels'])

In [43]:
etherscan_labels_dict = dfs['etherscan_account_labels'].set_index(
    'address').to_dict()['label']
sybil_list_labels_dict = dfs['sybil_list_account_labels'].set_index(
    'address').to_dict()['label']
tally_compound_labels_dict = dfs['tally_compound_account_labels'].set_index(
    'address').to_dict()['label']
tally_uniswap_labels_dict = dfs['tally_uniswap_account_labels'].set_index(
    'address').to_dict()['label']

print("There are {} addresses labeled by Etherscan".format(
    len(etherscan_labels_dict)))
print("There are {} addresses labeled by Sybil-List".format(
    len(sybil_list_labels_dict)))
print("There are {} addresses labeled by Tally - Compound".format(
    len(tally_compound_labels_dict)))
print("There are {} addresses labeled by Tally - Uniswap".format(
    len(tally_uniswap_labels_dict)))

# Update the Labels dictionaty with the labels, in order of priority!
labels = dict()
labels.update(etherscan_labels_dict)
labels.update(sybil_list_labels_dict)
labels.update(tally_compound_labels_dict)
labels.update(tally_uniswap_labels_dict)
print(f"Labels dictionary has {len(labels)} addresses.")

There are 26509 addresses labeled by Etherscan
There are 2783 addresses labeled by Sybil-List
There are 387 addresses labeled by Tally - Compound
There are 265 addresses labeled by Tally - Uniswap
Labels dictionary has 29733 addresses.


In [44]:
len(set(tally_compound_labels_dict.keys()).union(set(tally_uniswap_labels_dict.keys())))

627

In [45]:
tally_compound_labels_dict
tally_uniswap_labels_dict

{'0xe7925d190aea9279400cd9a005e33ceb9389cc2b': 'jessewldn',
 '0x8962285faac45a7cbc75380c484523bb7c32d429': 'Consensys',
 '0x88fb3d509fc49b515bfeb04e23f53ba339563981': 'Robert Leshner',
 '0x683a4f9915d6216f73d6df50151725036bd26c02': 'Gauntlet',
 '0xb55a948763e0d386b6defcd8070a522216ae42b1': 'hi_Reverie',
 '0x553f674dd7d102ad79c644103974a1cc53b62ac2': 'teemulaumhonkasalo.e…',
 '0x59a5493513ba2378ed57ae5ecfb8a027e9d80365': 'iamDCinvestor',
 '0xa2bf1b0a7e079767b4701b5a1d9d5700eb42d1d1': 'nkennethk',
 '0x13bdae8c5f0fc40231f0e6a4ad70196f59138548': 'Michigan Blockchain',
 '0xb0623c91c65621df716ab8afe5f66656b21a9108': 'John Palmer',
 '0x8d07d225a769b7af3a923481e1fdf49180e6a265': 'MonetSupply',
 '0x067b9bbbbc42d2ab9b9ab6bb62646daf7f344a76': 'lay2000lbs',
 '0x070341aa5ed571f0fb2c4a5641409b1a46b4961b': 'Franklin DAO',
 '0x47c125dee6898b6cb2379bcbafc823ff3f614770': 'blockchainucla',
 '0x1a13c53d516f6e8321e9903f224861e74bbd7aa0': 'will__price',
 '0xeff506a32b55d5c19847c1a4f8510c00280c27e5': 'Stanfo

In [46]:
# Making names more readable
for item in labels.items():
    if 'Compound Voting: ' in item[1]:
        labels[item[0]] = item[1].replace('Compound Voting: ', '')
    if item[1].startswith("Binance"):
        labels[item[0]] = "Binance"
    if item[1].startswith("Coinbase"):
        labels[item[0]] = "Coinbase"
    if item[1].startswith("Balancer"):
        labels[item[0]] = "Balancer"
    if item[1].startswith("Huobi"):
        labels[item[0]] = "Huobi"
    if item[1].startswith("SushiSwap"):
        labels[item[0]] = "SushiSwap"
    if item[1].startswith("Maker"):
        labels[item[0]] = "Maker"
    if item[1].startswith("Bancor"):
        labels[item[0]] = "Bancor"
    if item[1].startswith("Curve.fi"):
        labels[item[0]] = "Curve.fi"
    if item[1].startswith("Synthetix"):
        labels[item[0]] = "Synthetix"
    if item[1].startswith("KuCoin"):
        labels[item[0]] = "KuCoin"
    if item[1].startswith("Aave"):
        labels[item[0]] = "Aave"
    if item[1].startswith("Set:"):
        labels[item[0]] = "Set"
    if item[1].startswith("Autonomous Proposal"):
        labels[item[0]] = "Autonomous Proposal"
    if item[1].startswith("teemulaumhonkasalo"):
        labels[item[0]] = "teemulaumhonkasalo"
    # if item[1].startswith("Uniswap V2:"):
    #     labels[item[0]] = "Uniswap"
    # if item[1].startswith("Uniswap V3:"):
    #     labels[item[0]] = "Uniswap"
    # if item[1].startswith("Uniswap:"):
    #     labels[item[0]] = "Uniswap"
    if item[1].startswith("0x:"):
        labels[item[0]] = "0x"

labels['0xb933aee47c438f22de0747d57fc239fe37878dd1'] = "Wintermute"
labels['0x6626593c237f530d15ae9980a95ef938ac15c35c'] = "Gauntlet"
labels['0x1a9c8182c09f50c8318d769245bea52c32be35bc'] = "Uniswap V2: UNI Timelock"
labels['0x4b4e140d1f131fdad6fb59c13af796fd194e4135'] = "Uniswap Protocol: Treasury Vester"
labels['0x3d30b1ab88d487b0f3061f40de76845bec3f1e94'] = "Uniswap Protocol: Treasury Vester"
labels['0x090d4613473dee047c3f2706764f49e0821d256e'] = "Uniswap: Token Distributor"
labels['0x2775b1c75658be0f640272ccb8c72ac986009e38'] = "Compound: Reservoir"

In [47]:
file_dir = os.path.join(labels_dir, 'labels.json')
with open(file_dir, 'wt') as f:
    json.dump(labels, f, indent=4)
print("There are {} unique labels in the labels dictionary".format(len(labels)))

There are 29738 unique labels in the labels dictionary
